In [37]:
from collections import deque
from enum import Enum
import graphviz as gv

In [38]:
class char(Enum):
    VAR = 1
    SIGN = 2
def char_ind(a):
    return ord(a) - ord('a') if ord(a) > 90 else 26 + ord(a) - ord('A')

global count
count = 0
appeared = [-1] * 52
vert_to_char = [-1] * 52
f = open("input.txt", "r")
f2 = open("input_version2.txt")

In [39]:
def process_string(res, prod1, appeared1, vert_to_char1, nr):
    global count

    first_char = ord(res[0]) - ord('a')
    if appeared1[first_char] == -1:
        appeared1[first_char] = count
        vert_to_char1[count] = first_char
        prod1[count][0].append(nr)
        count += 1
    else:
        prod1[appeared1[first_char]][0].append(nr)

    ind = 4
    state = char.VAR
    signs = ['+', '-', '*', '/', '^', '%']

    while ind < len(res):
        while ind < len(res) and (res[ind] > 'z' or res[ind] < 'A' or 'Z' < res[ind] < 'a'):
            if res[ind] in signs:
                if state == char.VAR:
                    raise Exception("Bledne dane")
                state = char.VAR
            elif '0' <= res[ind] <= '9':
                if state == char.SIGN:
                    raise Exception("Bledne dane")
                while ind < len(res) and '0' <= res[ind] <= '9':
                    ind += 1
                state = char.SIGN
                continue
            ind += 1

        if ind < len(res):
            if state == char.SIGN:
                raise Exception("Bledne dane")
            other_char = char_ind(res[ind])
            if appeared1[other_char] == -1:
                appeared1[other_char] = count
                prod1[count][1].append(nr)
                count += 1
            else:
                prod1[appeared1[other_char]][1].append(nr)
            state = char.SIGN
        ind += 1
    if state == char.VAR:
        raise Exception("Bledne dane")



In [40]:
def process_input_v1():
    global count
    no_lines = int(f.readline().strip('\n'))
    print(no_lines)

    D = [[0 for _ in range(no_lines)] for _ in range(no_lines)]
    for i in range(no_lines):
        D[i][i] = 1


    prod = [[[], []] for _ in range(52)]
    for i in range(no_lines):
        res_i = f.readline().strip('\n')
        print(res_i)
        process_string(res_i, prod, appeared, vert_to_char, i)

    for i in range(count):
        for l in prod[i][0]:
            for r in prod[i][1]:
                D[l][r] = 1
                D[r][l] = 1
                
    ver_list = f.readline().strip('\n')
    print(ver_list)
    f.close()
    return D, ver_list


def process_input_v2():

    no_prod = int(f2.readline().strip('\n'))
    no_var = int(f2.readline().strip('\n'))
    D = [[0 for _ in range(no_prod)] for _ in range(no_prod)]
    for i in range(no_prod):
        D[i][i] = 1
    
    prod = [[[], []] for _ in range(no_var)]
    

    for j in range(no_prod):
        r = int(f2.readline().strip('\n'))
        l = int(f2.readline().strip('\n'))
        prod[l][0].append(j)
        R = f2.readline().strip('\n').split(' ')
        for var in R:
            prod[int(var)][1].append(j)
    

    for i in range(no_var):
        for l in prod[i][0]:
            for r in prod[i][1]:
                D[l][r] = 1
                D[r][l] = 1
    
    ver_list = f2.readline().strip('\n')
    print(ver_list)
    f2.close()
    return D, ver_list




In [41]:
def bfs_with_deletion(start_vertex, Graph):
    Q = deque()
    ways = [0] * len(Graph)
    Q.append(start_vertex)
    while len(Q):
        v = Q.popleft()
        ways[v] += 1
        for w, active in Graph[v]:
            if active:
                Q.append(w)

    for j in range(len(Graph[start_vertex])):
        if ways[Graph[start_vertex][j][0]] > 1:
            print(f"Deleting edge: {start_vertex+1} -> {Graph[start_vertex][j][0]+1}")
            Graph[start_vertex][j][1] = False

In [42]:
Gr = gv.Digraph(format='png',
                filename='graph.gv', engine='fdp')

vertexes = []
D, ver_list = process_input_v2()

print("Dependencies:")
print(D)

m = len(ver_list)
inner_edges = [0] * m

Foata = [[] for _ in range(m)]
in_class = [0] * m

G = [[] for _ in range(m)]

for pos, v in enumerate(ver_list):
    vertexes.append(char_ind(v))
    Gr.node(str(pos), v)
for i in range(m - 1):
    for j in range(i + 1, m):
        if D[vertexes[i]][vertexes[j]]:
            G[i].append([j, True])

for i in range(m - 1):
    bfs_with_deletion(i, G)

print("Graph:")
for i in range(m):
    for v, visit in G[i]:
        if visit:
            inner_edges[v] += 1
            Gr.edge(str(i), str(v), constraint='false')
            print(f"{i+1} -> {v+1}")

for k in range(m):
    Fi = []
    found = 0
    for i in range(m):
        if not in_class[i] and inner_edges[i] == 0:
            found = 1
            in_class[i] = k + 1
            Foata[k].append(i)

    for i in range(m):
        if in_class[i] == k + 1:
            for v, visit in G[i]:
                if visit:
                    inner_edges[v] -= 1
    if not found:
        break

print("Foata classes:")
for i in range(m):
    if len(Foata[i]):
        Foata[i].sort(key=lambda y: ver_list[y])
        print("[", end=' ')
        for v in Foata[i]:
            print(f"{ver_list[v]}", end=' ')
        print("]", end=' ')
    else:
        break
f.close()
f2.close()
Gr.render(".\\Graph", view = True)

acdcfbbe
Dependencies:
[[1, 0, 1, 0, 0, 1], [0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1, 1], [0, 0, 0, 1, 0, 1], [0, 1, 1, 0, 1, 0], [1, 0, 1, 1, 0, 1]]
Deleting edge: 1 -> 4
Deleting edge: 1 -> 5
Deleting edge: 2 -> 5
Deleting edge: 2 -> 8
Deleting edge: 6 -> 8
Graph:
1 -> 2
2 -> 4
3 -> 5
4 -> 5
4 -> 8
6 -> 7
7 -> 8
Foata classes:
[ a b d ] [ b c ] [ c ] [ e f ] 

'Graph.png'